## 01 - nc-extractor.ipynb

### Table of content

[01-01]: Iterate over raw data

[01-02]: Save extracted data

In [5]:
import netCDF4 as nc

In [26]:
fn_ = '../_data/waterhypernet/hypstar/processed20230317/LVES/2023/05/06/SEQ20230506T050252/HYPERNETS_W_LVES_L2A_REF_20230506T0502_20230509T0031_v1.0.nc'
ds_ = nc.Dataset(fn_)
print(ds_.__dict__)

{'type': 'dataset', 'abstract': 'The HYPERNETS project (Horizon 2020 research and innovation, grant agreement No 775983) has the overall aim to provide high quality in situ measurements tosupport the (visible/SWIR) optical Copernicus products. Therefore a new multi-head hyperspectral spectroradiometer dedicated to land and water surface reflectance validation with instrument pointing capabilities and embedded calibration device has been established. The instrument has been deployed at 24 sites covering a range of water and land types and a range of climatic andlogistic conditions (www.hypernets.eu).', 'conventions': 'CFv72, NVS2.0', 'format_version': 'v01.0', 'netcdf_version': '1.6', 'processor_name': 'hypernets_processor', 'processor_version': '0.1', 'processor_atbd': 'TBD', 'product_name': 'HYPERNETS_W_LVES_L2A_REF_20230506T0502_20230509T0031_v1.0', 'data_created': '05/09/2023, 00:31:52', 'references': 'https://hypernets-processor.readthedocs.io/en/latest/', 'source': 'surface observ

In [27]:
print(ds_.variables.keys())

dict_keys(['wavelength', 'bandwidth', 'viewing_azimuth_angle', 'viewing_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle', 'acquisition_time', 'series_id', 'vaa_ref', 'vaa_abs', 'vaa_ask', 'quality_flag', 'reflectance', 'water_leaving_radiance', 'reflectance_nosc', 'u_rel_random_water_leaving_radiance', 'u_rel_systematic_water_leaving_radiance', 'corr_random_water_leaving_radiance', 'corr_systematic_water_leaving_radiance', 'u_rel_random_reflectance', 'u_rel_systematic_reflectance', 'corr_random_reflectance', 'corr_systematic_reflectance', 'u_rel_random_reflectance_nosc', 'u_rel_systematic_reflectance_nosc', 'corr_random_reflectance_nosc', 'corr_systematic_reflectance_nosc', 'epsilon', 'u_rel_random_epsilon', 'u_rel_systematic_epsilon', 'rhof', 'fresnel_wind', 'reflectance_std', 'water_leaving_radiance_std', 'reflectance_nosc_std'])


#### [01-01]: Iterate over raw data

In [29]:
# _nc_files % find ../waterhypernet -type f -name "*.nc" -exec cp {} .\;

import pandas as pd
import os

directory_path = "/Volumes/RND_1/DataScience214386/LTT0.00.027 - Data Science in Remote Sensing/2023/_project/_data/_nc_files"

df_all_metadata = pd.DataFrame()
df_all_data = pd.DataFrame()
for filename in os.listdir(directory_path):
    filepath = os.path.join(directory_path, filename)
    if os.path.isfile(filepath) and filename[0] != '.':
        ds = nc.Dataset(filepath)
        df_all_metadata__ = pd.DataFrame(ds.__dict__, index=[0])
        metadata = ['data_created', 'sequence_id', 'system_id', 'site_id', 'site_latitude', 'site_longitude', 'source_file', 'system_temperature', 'system_pressure', 'illuminance', 'system_relative_humidity','fresnel_option', 'similarity_waveref', 'similarity_wavethres', 'similarity_wavelen1', 'similarity_wavelen2', 'similarity_alpha', 'title', 'instrument_calibration_file_rad', 'IRR_acceleration_x_mean', 'IRR_acceleration_x_std', 'ss_res', 'product_level']
        df_all_metadata__ = df_all_metadata__[metadata]
        sequence_id = str(df_all_metadata__['sequence_id'][0])
        system_id = str(df_all_metadata__['system_id'][0])
        site_id = str(df_all_metadata__['site_id'][0])
        source_file = str(df_all_metadata__['source_file'][0])
        data_created = pd.to_datetime(str(df_all_metadata__['data_created'][0]), format='%m/%d/%Y, %H:%M:%S')
        df_all_metadata__['data_created'] = data_created

        df_all_data__ = pd.DataFrame()
        #variables = ['wavelength', 'bandwidth', 'viewing_azimuth_angle', 'viewing_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle', 'acquisition_time', 'series_id', 'vaa_ref', 'vaa_abs', 'vaa_ask', 'quality_flag', 'reflectance', 'water_leaving_radiance', 'reflectance_nosc', 'u_rel_random_water_leaving_radiance', 'u_rel_systematic_water_leaving_radiance', 'corr_random_water_leaving_radiance', 'corr_systematic_water_leaving_radiance', 'u_rel_random_reflectance', 'u_rel_systematic_reflectance', 'corr_random_reflectance', 'corr_systematic_reflectance', 'u_rel_random_reflectance_nosc', 'u_rel_systematic_reflectance_nosc', 'corr_random_reflectance_nosc', 'corr_systematic_reflectance_nosc', 'epsilon', 'u_rel_random_epsilon', 'u_rel_systematic_epsilon', 'rhof', 'fresnel_wind', 'reflectance_std', 'water_leaving_radiance_std', 'reflectance_nosc_std']
        variables = ['wavelength', 'reflectance']
        for variable in variables:
            var_array_ = ds.variables[f'{variable}']
            if (var_array_.ndim == 1 and len(var_array_) == 1):
                df_all_metadata__[f'{variable}'] = ds.variables[f'{variable}'][:][0]
            elif (var_array_.ndim == 1 and len(var_array_) > 1):
                df_all_data__[f'{variable}'] = ds.variables[f'{variable}'][:]
            elif (var_array_.ndim == 2 and var_array_.shape[1] == 1):
                df_all_data__[f'{variable}'] = ds.variables[f'{variable}'][:]
            else:
                print(f'skipped: {variable}')

        df_all_metadata = pd.concat([df_all_metadata, df_all_metadata__], ignore_index=True)
        df_all_data__['data_created'] = data_created
        df_all_data__['sequence_id'] = sequence_id
        df_all_data__['system_id'] = system_id
        df_all_data__['site_id'] = site_id
        df_all_data__['source_file'] = source_file
        df_all_data = pd.concat([df_all_data, df_all_data__], ignore_index=True)

In [30]:
df_all_metadata.head(2)

,data_created,sequence_id,system_id,site_id,site_latitude,site_longitude,source_file,system_temperature,system_pressure,illuminance,...,similarity_wavethres,similarity_wavelen1,similarity_wavelen2,similarity_alpha,title,instrument_calibration_file_rad,IRR_acceleration_x_mean,IRR_acceleration_x_std,ss_res,product_level
0,2023-05-20 16:16:40,SEQ20230520T140039,HYPSTAR_122241,LVES,58.211082,26.107643,SEQ20230520T140039,23.61,1027.81,40113.9,...,0.05,720,780,2.35,HYPSTAR Water network dataset of spectral surf...,HYPERNETS_CAL_HYPSTAR_122241_RAD_v0.1.nc,-9.099373817443848,0.03329671360552311,0.000988841544155856,W_L2A
1,2023-05-20 17:46:25,SEQ20230520T153043,HYPSTAR_122241,LVES,58.211082,26.107643,SEQ20230520T153043,24.46,1026.99,5208.3,...,0.05,720,780,2.35,HYPSTAR Water network dataset of spectral surf...,HYPERNETS_CAL_HYPSTAR_122241_RAD_v0.1.nc,-9.091597557067871,0.03200073167681694,0.0039103342352075745,W_L2A


In [31]:
df_all_data.head(2)

,wavelength,reflectance,data_created,sequence_id,system_id,site_id
0,350.45,-0.051876,2023-05-20 16:16:40,SEQ20230520T140039,HYPSTAR_122241,LVES
1,350.91,-0.050752,2023-05-20 16:16:40,SEQ20230520T140039,HYPSTAR_122241,LVES


#### [01-02]: Save extracted data

In [32]:
df_all_metadata = df_all_metadata.sort_values(by='data_created')
df_all_metadata.to_csv("/Volumes/RND_1/DataScience214386/LTT0.00.027 - Data Science in Remote Sensing/2023/_project/_data/_nc_extracts/metadata_extracted.csv", index=False)

df_all_data = df_all_data.sort_values(by='data_created')
df_all_data.to_csv("/Volumes/RND_1/DataScience214386/LTT0.00.027 - Data Science in Remote Sensing/2023/_project/_data/_nc_extracts/data_extracted.csv", index=False)